In [29]:
from keras.layers import Dense, Dropout
from keras.models import Sequential
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd

df = pd.read_csv("https://projects.fivethirtyeight.com/soccer-api/club/spi_matches.csv").dropna()
df = df[(df.league=='Barclays Premier League') | (df.league=='English League Championship') | (df.league=='Major League Soccer')]
df['spi_margin'] = np.abs(df['spi1'] - df['spi2'])
df['margin'] = np.abs(df['score1'] - df['score2'])
df['totgoals'] = df['score1'] + df['score2']

res = []

for x in df['totgoals']:
  if x > 0:
    res.append("GOALS")
  else:
    res.append("SCORELESS")

df['goals'] = res

In [31]:
def load_model():
  model = Sequential()
  model.add(Dense(16, activation='relu'))
  model.add(Dense(2, activation='softmax'))
  model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
  return model

def to_cat(array):
  y = to_categorical(np.asarray(pd.factorize(array)[0]))
  return y

def to_df(array):
  df = pd.DataFrame(array).reset_index(drop=True)
  return df

In [32]:
model = load_model()

In [33]:
X = df[['score1','score2']].reset_index(drop=True)
y = df[['goals']].copy()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=6)

In [34]:
y_train = to_cat(y_train['goals'])
y_train = to_df(y_train)
y_test = to_cat(y_test['goals'])
y_test = to_df(y_test)

In [35]:
model.fit(X_train, y_train, epochs=40, batch_size=128, verbose=1, validation_split=0.1)

Epoch 1/40
29/29 [==============================] - 1s 9ms/step - loss: 0.4120 - accuracy: 0.9347 - val_loss: 0.2771 - val_accuracy: 0.9538
Epoch 2/40
29/29 [==============================] - 0s 3ms/step - loss: 0.2576 - accuracy: 0.9341 - val_loss: 0.1850 - val_accuracy: 0.9538
Epoch 3/40
29/29 [==============================] - 0s 2ms/step - loss: 0.1878 - accuracy: 0.9286 - val_loss: 0.1420 - val_accuracy: 0.9538
Epoch 4/40
29/29 [==============================] - 0s 3ms/step - loss: 0.1572 - accuracy: 0.9272 - val_loss: 0.1180 - val_accuracy: 0.9538
Epoch 5/40
29/29 [==============================] - 0s 2ms/step - loss: 0.1260 - accuracy: 0.9404 - val_loss: 0.1031 - val_accuracy: 0.9538
Epoch 6/40
29/29 [==============================] - 0s 3ms/step - loss: 0.1206 - accuracy: 0.9315 - val_loss: 0.0935 - val_accuracy: 0.9538
Epoch 7/40
29/29 [==============================] - 0s 3ms/step - loss: 0.1134 - accuracy: 0.9312 - val_loss: 0.0858 - val_accuracy: 0.9538
Epoch 8/40
29/29 [==

In [36]:
predictions = model.predict(X_test).round(0)
pred2 = model.predict(X_train)

In [37]:
loss2, acc2 = model.evaluate(y_train, pred2)
loss, acc = model.evaluate(y_test, predictions)

33/33 [==============================] - 0s 1ms/step - loss: 0.3788 - accuracy: 0.9211


In [38]:
print("Train Prediction Accuracy: %.2f" % (acc2 * 100) + "%")
print("Test Prediction Accuracy: %.2f" % (acc * 100) + "%")

Train Prediction Accuracy: 93.44%
Test Prediction Accuracy: 92.11%
